In [16]:
import json
from collections import defaultdict
train_file = "./duee_fin_train.json"
trigger_words = defaultdict(set)
with open(train_file, 'r',encoding='utf-8') as f:
    train_lines = f.readlines()

no_event = 0
for line in train_lines:
    line = json.loads(line.strip())
    if "event_list" not in line:
        # print(line)
        no_event += 1
        continue
    for event in line['event_list']:
        trigger_words[event['event_type']].add(event['trigger'])



In [9]:
print("触发词:",dict(trigger_words))
print("事件类型:",trigger_words.keys())

触发词: {'质押': {'股份质押', '解除质押', '出质', '延期', '到期', '质押'}, '股份回购': {'股票回购', '回购', '购回', '回股', ')回购', '购入'}, '解除质押': {'质押解除', '届满', '解押', '解质', '解质押', '解除', '延期', '延期购回', '到期', '质押式回购', '质押', '质押股份', '股权解质', '解除质押', '解除了质押'}, '被约谈': {'被约谈', '约谈', '谈话'}, '企业收购': {'出售', '受让', '收购', '拿下', '合并', '划转', '取得', '并购', '重组', '收購', '投资', '出让', '认购', '转让', '购入', '购买'}, '股东增持': {'持股量增加', '买入', '增持', '增加', '增资', '购买'}, '高管变动': {'推选', '新增', '不再代行', '退居', '转战', '代任', '离任', '任', '变更', '履新', '信维通信', '任免', '辞任', '不再兼任', '入职', '聘请', '不再担任', '新任', '免去', '解聘', '升任', '主持', '新添', '转任', '离职', '更换', '任命', '任职', '变动', '履职', '拟任', '增聘', '退任', '辞去', '辞职', '加盟', '接手', '招来', '升职', '现任', '重选', '调任', '将为', '高管变动', '兼任', '提名', '继任', '罢免', '履行', '聘任', '当选', '辞呈', '接任', '选举', '继续担任', '去世', '不再负责', '上任', '调整', '代行', '接棒', '就任', '退休', '委任', '逝世', '卸任', '将任', '出任', '聘', '补选', '接替', '大换血', '退出', '担任'}, '中标': {'连中三标', '中标', '承建', '招标', '中选'}, '公司上市': {'上市', '发行股票', '上会', '招股书', '终止上市', '退市', '冲刺', '发行', '闯关', '摘牌', 'IPO', '登陆', '暂停

In [2]:
print("总样本量",len(lines))
print("没有事件的样本量:",no_event)

总样本量 7047
没有事件的样本量: 897


In [17]:
with open("duee_fin_event_schema.json","r" ,encoding="utf-8") as f:
    schema_lines = f.readlines()

events = defaultdict(list)
for line in schema_lines:
    line = json.loads(line.strip())
    for event in line['role_list']:
        events[line['event_type']].append(event['role'])
print("事件类型及角色:")
for i,(k,v) in enumerate(events.items()):
    print(f"事件类型{i}:{k} \t角色数量及类型:{len(v)}\t{v}")

事件类型及角色:
事件类型0:解除质押 	角色数量及类型:9	['质押方', '披露时间', '质权方', '质押物', '质押股票/股份数量', '事件时间', '质押物所属公司', '质押物占总股比', '质押物占持股比']
事件类型1:股份回购 	角色数量及类型:7	['回购方', '披露时间', '回购股份数量', '每股交易价格', '占公司总股本比例', '交易金额', '回购完成时间']
事件类型2:股东减持 	角色数量及类型:9	['股票简称', '披露时间', '交易股票/股份数量', '每股交易价格', '交易金额', '交易完成时间', '减持方', '减持部分占所持比例', '减持部分占总股本比例']
事件类型3:亏损 	角色数量及类型:5	['公司名称', '披露时间', '财报周期', '净亏损', '亏损变化']
事件类型4:中标 	角色数量及类型:6	['中标公司', '中标标的', '中标金额', '招标方', '中标日期', '披露日期']
事件类型5:高管变动 	角色数量及类型:8	['高管姓名', '任职公司', '高管职位', '事件时间', '变动类型', '披露日期', '变动后职位', '变动后公司名称']
事件类型6:企业破产 	角色数量及类型:5	['破产公司', '披露时间', '债务规模', '破产时间', '债权人']
事件类型7:股东增持 	角色数量及类型:9	['股票简称', '披露时间', '交易股票/股份数量', '每股交易价格', '交易金额', '交易完成时间', '增持方', '增持部分占所持比例', '增持部分占总股本比例']
事件类型8:被约谈 	角色数量及类型:4	['公司名称', '披露时间', '被约谈时间', '约谈机构']
事件类型9:企业收购 	角色数量及类型:6	['收购方', '披露时间', '被收购方', '收购标的', '交易金额', '收购完成时间']
事件类型10:公司上市 	角色数量及类型:8	['上市公司', '证券代码', '环节', '披露时间', '发行价格', '事件时间', '市值', '募资金额']
事件类型11:企业融资 	角色数量及类型:7	['投资方', '披露时间', '被投资方', '融资金额', '融资轮次', '事件时间', '领投方']

In [45]:
# 找寻每个事件必须存在及关键的role
keyroles = {}
breaks = False
for line in train_lines:
    line = json.loads(line.strip())
    if "event_list" not in line:
        continue
    event_list = line['event_list']
    text = line['text']
    # if breaks:
    #     break
    for event in event_list:
        event_type = event['event_type']
        arguments = event['arguments']
        roles = set([argument['role'] for argument in arguments])
        if event_type not in keyroles:
            keyroles[event_type] = roles
        else:
            if event_type == "质押" and "质押物所属公司" not in roles:
                print(event_type,keyroles[event_type],roles,text)
            if len(keyroles[event_type]&roles)==0:
                breaks = True
                break
            keyroles[event_type] = keyroles[event_type]&(roles)

质押 {'质押物所属公司'} {'质押方'} 原标题：金凰珠宝百亿质押黄金风波续集：民生信托东莞信托发布告投资者书    来源：证券时报
金凰珠宝违约事件持续发酵，深陷其中的金融机构也被置于风口浪尖。面对已经到来及即将到来的兑付压力，东莞信托与民生信托先后公开发布情况说明，陈述对于此次纠纷的观点及立场。
解析担保方式
民生信托在致投资者的说明函中表示，“金凰相关项目的成立、运作、管理各环节，我司均依法合规履行受托人职责，严格遵守信托文件约定，为实现受益人的最大利益处理信托事务。我司承办金凰项目的初衷，在于积极响应国家政策，服务支持实体经济，为融资方参与湖北省国有企业改革以及氢燃料电池研发制造提供资金支持。
相关信托计划采取了‘保险公司承保+实物黄金质押’的核心担保措施，由保险公司出具条款严密的保险单，我司作为受托人，为保险单的第一受益人。保险单特别约定：标的黄金的质量和重量由保险人承保，在受益人需要处置标的黄金时，保险人有义务对受益人予以配合，保险人交付给受益人的标的黄金应经过双方认可的具有黄金鉴定资质的第三方检测机构检测合格（抽样检测）。如质量和重量不符合保险单约定，即视同发生保险事故，由保险人承担全部赔偿责任。
保险公司作为保险人主持了质押黄金交付的全过程，包括但不限于质押黄金的出库、检测、运输、清点、封存过程。目前保险公司承保的质押黄金封存于银行保管箱。”
质押 {'质押物所属公司'} {'质押物占总股比', '质押方', '质权方', '事件时间', '披露时间', '质押物占持股比', '质押股票/股份数量'} 挖贝网 4月18日消息，中公教育（002607）今日发布公告，公司控股股东、实际控制人鲁忠芳向中国民生银行股份有限公司北京分行质押5000万股，用于为北京汇友致远投资中心（有限合伙）融资提供质押担保。
公告显示，中公教育本次质押股数5000万股，占其所持股份比例为1.96%，占公司总股本比例0.81%，质押日期自2020年4月16日至2023年4月8日，质权人为中国民生银行股份有限公司北京分行。
截至本公告披露日，鲁忠芳及其一致行动人所持质押股份情况如下：李永新、鲁忠芳、北京中公未来信息咨询中心（有限合伙）合计持股数量3,761,964,381股，合计持股比例61%，合计占其所持股份比例31.58%，合计占公司总股本比例19.27%。
公司2019

In [30]:
keyroles

{'质押': set(),
 '股份回购': set(),
 '解除质押': set(),
 '被约谈': set(),
 '企业收购': set(),
 '股东增持': set(),
 '高管变动': set(),
 '中标': set(),
 '公司上市': {'环节'},
 '企业融资': set(),
 '亏损': set(),
 '股东减持': set(),
 '企业破产': set()}